In [57]:
import pandas as pd

column_names = ['itunes_id', 'episode_md', 'sentences', 'advertisement']
new_df = pd.DataFrame(columns=column_names)

def add_row(sentences, advertisement, itunes_id, episode_md):
    # append no_advert section to the dataframe 
    new_df.loc[len(new_df)] = {"sentences": sentences, 
                               "advertisement" : advertisement, 
                               'itunes_id': itunes_id, 
                               'episode_md': episode_md}
    
def check_csv(itunes_id, episode_md, df):
    word_window = []
    last_was_advertisement = -1 # initialized (true/false)


    for index, row in df.iterrows():
        # init the window tracker
        if last_was_advertisement == -1: 
            last_was_advertisement = row['advertisement']
            word_window.append(str(row['word']))
            
        # change from no_advert to advert
        elif row['advertisement'] == 1 and last_was_advertisement == 0: 
            # sentences in the advert/no_advert window
            sentences = ' '.join(word_window) 
            # reset the window, so it can receive new words 
            word_window = []
            # set to advert window
            last_was_advertisement = 1
            word_window.append(str(row['word']))

            add_row(sentences, 0, itunes_id, episode_md)

        # capture change from advert to no_advert
        elif row['advertisement'] == 0 and last_was_advertisement == 1:
            sentences = ' '.join(word_window) 
            word_window = []
            last_was_advertisement = 0
            word_window.append(str(row['word']))

            add_row(sentences, 1, itunes_id, episode_md)

        # last index, finish df 
        elif index == len(df) - 1: 
            # append the last known values 
            sentences =  ' '.join(word_window)
            add_row(sentences, last_was_advertisement, itunes_id, episode_md)
        
        # nothing of the above => continue execution 
        else: 
            word_window.append(str(row['word']))

def add_advert_column(itunes_id, episode_md, path): 
    df = pd.read_csv(path)

    # create a new column 'advertisement' with default value 0
    df['advertisement'] = 0

    # loop through each row of the dataframe
    for index, row in df.iterrows():
    # try to parse the speaker column to integer
        try:
            speaker_int = int(row['speaker'])
        # if parsing fails, set the 'advertisement' column to 1
        except ValueError:
            df.at[index, 'advertisement'] = 1
    
    check_csv(itunes_id, episode_md, df)


In [61]:
import os 

counter = 0

folder_path = './data/podscribe/'

folder_contents = os.listdir(folder_path)

subfolders = [f for f in folder_contents if os.path.isdir(os.path.join(folder_path))]

for subfolder in subfolders: 
    files = os.listdir('./data/podscribe/' + subfolder)

    csv_files = [f for f in files if f.endswith('.csv')]
    
    for csv_file in csv_files: 
        path = os.path.join(folder_path, subfolder, csv_file)
        itunes_id =  csv_file.split("_")[1]
        episode_md = csv_file.split("_")[2][:-4]

        counter +=1
        if counter % 200 == 0: 
            print(counter) 

        try:         
            add_advert_column(itunes_id, episode_md, path)
        except Exception as e: 
            print("error")

200
400
600
800
1000
1200
error
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800


NotADirectoryError: [Errno 20] Not a directory: './data/podscribe/.DS_Store'

In [65]:
new_df['sentences'][len(new_df)-1]

"Done. What Are You using for it's, it's a new client. I Use it for everything. What's the nastiest thing you look at on the internet? Adam? I don't really use the, I mean, I don't, I don't even really look at porn that much anymore. You don't? No. Not that much. What that means, it's Like, I don't know. I live with my girlfriend, like, I don't know. She's home. She's home. I gotta, I'm gonna go like Just She was gone more. You Would? Yeah. She's been outta town this week. I've had a nice time. How many times have you looked at porn this week? Probably three. Three times. Three Times. What's your refractory? What's refractory? You know that? Oh, for like reloading? Yeah. Oh, I'm, I can shoot heavy 15 to 20 minutes later. 15 to 20 minutes. Yeah. Ropes Peter North. If I jack off, I probably, I can't get horny for like three days. Really? Yeah. Yeah. I don't really use the internet for anything anymore Either. Yeah. I like, I use Wikipedia a lot. I don't even use Wikipedia. I make AI art 

In [66]:
new_df.to_csv('clean.csv', index=False)